<a href="https://colab.research.google.com/github/jimchen2/EECS-149-Final-Project/blob/main/Training_Yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Clone the repos**

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!git clone https://github.com/EscVM/OIDv4_ToolKit

# **Install Dependencies**

In [ ]:
!cd OIDv4_ToolKit && pip install -r requirements.txt
!cd yolov5 && pip install -r requirements.txt

# **Download datasets from OpenImages**

Type Y Manually

In [ ]:
!yes| python3 OIDv4_ToolKit/main.py downloader --classes "Tin_can" --type_csv train
!yes| python3 OIDv4_ToolKit/main.py downloader --classes "Tin_can" --type_csv test
!yes| python3 OIDv4_ToolKit/main.py downloader --classes "Tin_can" --type_csv validation


		   ___   _____  ______            _    _    
		 .'   `.|_   _||_   _ `.         | |  | |   
		/  .-.  \ | |    | | `. \ _   __ | |__| |_  
		| |   | | | |    | |  | |[ \ [  ]|____   _| 
		\  `-'  /_| |_  _| |_.' / \ \/ /     _| |_  
		 `.___.'|_____||______.'   \__/     |_____|
	

             _____                    _                 _             
            (____ \                  | |               | |            
             _   \ \ ___  _ _ _ ____ | | ___   ____  _ | | ____  ____ 
            | |   | / _ \| | | |  _ \| |/ _ \ / _  |/ || |/ _  )/ ___)
            | |__/ / |_| | | | | | | | | |_| ( ( | ( (_| ( (/ /| |    
            |_____/ \___/ \____|_| |_|_|\___/ \_||_|\____|\____)_|    
                                                          
        
    [INFO] | Downloading Tin can.
   [ERROR] | Missing the class-descriptions-boxable.csv file.
...145%, 0 MB, 68756 KB/s, 0 seconds passed
[DOWNLOAD] | File class-descriptions-boxable.csv downloaded into OID/csv_folder/c

# **Adjust folders**

In [29]:
!mkdir -p /content/images/train /content/labels/train
!mkdir -p /content/images/test /content/labels/test
!mkdir -p /content/images/validation /content/labels/validation

!cp -r OIDv4_ToolKit/OID/Dataset/train/'Tin can'/Label/* /content/labels/train
!cp -r OIDv4_ToolKit/OID/Dataset/test/'Tin can'/Label/* /content/labels/test
!cp -r OIDv4_ToolKit/OID/Dataset/validation/'Tin can'/Label/* /content/labels/validation

!cp -r OIDv4_ToolKit/OID/Dataset/train/'Tin can'/* /content/images/train
!cp -r OIDv4_ToolKit/OID/Dataset/test/'Tin can'/* /content/images/test
!cp -r OIDv4_ToolKit/OID/Dataset/validation/'Tin can'/* /content/images/validation


In [30]:
!rm -rf images/test/Label images/validation/Label images/train/Label

# **Adjust Labels**

In [31]:
import os

def convert_bbox(image_width, image_height, x_min, y_min, x_max, y_max):
    # Conversion logic remains the same
    dw = 1./image_width
    dh = 1./image_height
    x = (x_min + x_max) / 2.0
    y = (y_min + y_max) / 2.0
    w = x_max - x_min
    h = y_max - y_min
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]

def process_labels(logdir, image_width=1280, image_height=720, class_id_for_tin_can=0):
    # Iterate through each file in the specified directory
    for filename in os.listdir(logdir):
        if filename.endswith('.txt'):
            file_path = os.path.join(logdir, filename)
            with open(file_path, 'r') as file:
                lines = file.readlines()

            new_lines = []
            for line in lines:
                if line.startswith('Tin can'):
                    # Extract coordinates and convert them
                    coordinates = line[7:].strip().split()
                    x_min, y_min, x_max, y_max = map(float, coordinates)
                    yolo_format = convert_bbox(image_width, image_height, x_min, y_min, x_max, y_max)
                    new_lines.append(f"{class_id_for_tin_can} {' '.join(map(str, yolo_format))}\n")

            with open(file_path, 'w') as file:
                file.writelines(new_lines)

In [32]:
process_labels('/content/labels/test')
process_labels('/content/labels/validation')
process_labels('/content/labels/train')

In [35]:
!cat labels/test/051834c6d1a060ca.txt

0 0.31444770000000005 0.7982947555555556 0.24549299999999996 0.9800220444444444


# **Create Index of Training Images**

In [43]:
yaml_content = '''
path: /content # root directory path of the dataset
train: images/train  # path to training images
val: images/validation  # path to validation images

# number of classes
nc: 1

# class names
names: ['Tin can']
'''

with open('/content/dataset.yaml', 'w') as file:
    file.write(yaml_content)


# **Structures**
<details>

```
├── images
│   ├── test
│   ├── train
│   └── validate
├── labels
│   ├── test
│   ├── train
│   ├── train.cache
│   ├── validate
│   └── validate.cache
├── OIDv4_ToolKit
│   ├── classes.txt
│   ├── images
│   ├── LICENSE
│   ├── main.py
│   ├── modules
│   ├── OID
│   ├── README.md
│   └── requirements.txt
├── script.sh
├── script_to_format.sh
├── tincan.yaml
└── yolov5
    ├── benchmarks.py
    ├── CITATION.cff
    ├── classify
    ├── CONTRIBUTING.md
    ├── data
    ├── detect.py
    ├── env
    ├── export.py
    ├── hubconf.py
    ├── LICENSE
    ├── models
    ├── __pycache__
    ├── README.md
    ├── README.zh-CN.md
    ├── requirements.txt
    ├── runs
    ├── segment
    ├── setup.cfg
    ├── train.py
    ├── tutorial.ipynb
    ├── utils
    ├── val.py
    └── yolov5s.pt
```
</details>


In [ ]:
!apt install tree && tree -L 2

# **Train on Images**

In [2]:
!cd yolov5 && python train.py --img 640 --batch 16 --epochs 200 --data /content/dataset.yaml --weights yolov5s.pt --cache

2023-11-28 07:12:41.160154: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 07:12:41.160200: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 07:12:41.160225: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8,

# **Download the exp and weights**

In [3]:
!zip -r yolov5_runs_train.zip yolov5/runs/train/exp6 # Change to latest exp
from google.colab import files
files.download('yolov5_runs_train.zip')


  adding: yolov5/runs/train/exp6/ (stored 0%)
  adding: yolov5/runs/train/exp6/results.png (deflated 7%)
  adding: yolov5/runs/train/exp6/events.out.tfevents.1701155563.d0089f8eec78.17447.0 (deflated 38%)
  adding: yolov5/runs/train/exp6/P_curve.png (deflated 16%)
  adding: yolov5/runs/train/exp6/weights/ (stored 0%)
  adding: yolov5/runs/train/exp6/weights/last.pt (deflated 9%)
  adding: yolov5/runs/train/exp6/weights/best.pt (deflated 9%)
  adding: yolov5/runs/train/exp6/confusion_matrix.png (deflated 39%)
  adding: yolov5/runs/train/exp6/F1_curve.png (deflated 15%)
  adding: yolov5/runs/train/exp6/labels_correlogram.jpg (deflated 37%)
  adding: yolov5/runs/train/exp6/train_batch2.jpg (deflated 3%)
  adding: yolov5/runs/train/exp6/val_batch0_pred.jpg (deflated 8%)
  adding: yolov5/runs/train/exp6/hyp.yaml (deflated 45%)
  adding: yolov5/runs/train/exp6/results.csv (deflated 84%)
  adding: yolov5/runs/train/exp6/train_batch1.jpg (deflated 3%)
  adding: yolov5/runs/train/exp6/val_batch

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Add Validation**

In [7]:
!python yolov5/val.py --weights yolov5/runs/train/exp6/weights/best.pt --data /content/dataset.yaml --img 640 --conf 0.001 --iou 0.65 --half

val: data=/content/dataset.yaml, weights=['yolov5/runs/train/exp6/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.65, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5/runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v7.0-247-g3f02fde Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/labels/validation.cache... 31 images, 0 backgrounds, 18 corrupt: 100% 31/31 [00:00<?, ?it/s]
val: WARNING ⚠️ /content/images/validation/0804e479f3848a5a.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1935]
val: WARNING ⚠️ /content/images/validation/0dfd223b59953d5d.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0667]
val: WARN

# **Try the new model on images**
<img src="https://www.shutterstock.com/image-photo/norway-august-4-2019-crushed-600nw-1475711459.jpg"></img>\

# The original model sees a fire hydrant while the trained model sees a tin can

In [11]:
!wget -O 1.png https://www.shutterstock.com/image-photo/norway-august-4-2019-crushed-600nw-1475711459.jpg
!python yolov5/detect.py --weights yolov5/runs/train/exp6/weights/best.pt --img 640 --conf 0.25 --source 1.png
!python yolov5/detect.py --weights yolov5/yolov5s.pt --img 640 --conf 0.25 --source 1.png


--2023-11-28 07:38:19--  https://www.shutterstock.com/image-photo/norway-august-4-2019-crushed-600nw-1475711459.jpg
Resolving www.shutterstock.com (www.shutterstock.com)... 13.224.250.68, 13.224.250.51, 13.224.250.60, ...
Connecting to www.shutterstock.com (www.shutterstock.com)|13.224.250.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81828 (80K) [image/jpeg]
Saving to: ‘1.png’

1.png               100%[===================>]  79.91K  --.-KB/s    in 0.007s  

2023-11-28 07:38:19 (11.1 MB/s) - ‘1.png’ saved [81828/81828]

detect: weights=['yolov5/runs/train/exp6/weights/best.pt'], source=1.png, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=Fals